#  Sentimental Analysis on Tweets

## 1. Getting Data Ready

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import re

In [2]:
fr = pd.read_csv("combine_kaggle_dataset.csv",encoding='latin1')
fr.head()

,Tweet Posted Time (UTC),Tweet Content,Tweet Location
0,01/03/2020,Why .Indians to prayer Iran a polluted http://...,Bihar
1,01/03/2020,Even death is âMade in Chinaâ now #CoronaV...,Goa
2,01/03/2020,One of the most widely read published stories...,Goa
3,01/03/2020,Hahahh RAW mission in Iran is failed due to C...,Punjab
4,01/03/2020,#BreakingNews: in show @sawaalwithamber on @SA...,Punjab


In [3]:
fr.tail()

,Tweet Posted Time (UTC),Tweet Content,Tweet Location
814792,09/06/2020,How to use contact lenses safely during these ...,Dehradun
814793,09/06/2020,Digital Justice System in India: Overview & Re...,Dehradun
814794,09/06/2020,#IndiaFightsCorona Stigma around #COVID19 can ...,Dehradun
814795,09/06/2020,These 8 steps will help revive #indianeconomy ...,Dehradun
814796,09/06/2020,Continuous improvement in COVID-19 doubling r...,Dehradun


In [4]:
fr['Tweet Content']

0         Why .Indians to prayer Iran a polluted http://...
1         Even death is âMade in Chinaâ now #CoronaV...
2         One of the most widely read  published stories...
3         Hahahh  RAW mission in Iran is failed due to C...
4         #BreakingNews: in show @sawaalwithamber on @SA...
                                ...                        
814792    How to use contact lenses safely during these ...
814793    Digital Justice System in India: Overview & Re...
814794    #IndiaFightsCorona Stigma around #COVID19 can ...
814795    These 8 steps will help revive #indianeconomy ...
814796    Continuous improvement in COVID-19 doubling  r...
Name: Tweet Content, Length: 814797, dtype: object

In [5]:
fr.describe

<bound method NDFrame.describe of        Tweet Posted Time (UTC)  \
0                   01/03/2020   
1                   01/03/2020   
2                   01/03/2020   
3                   01/03/2020   
4                   01/03/2020   
...                        ...   
814792              09/06/2020   
814793              09/06/2020   
814794              09/06/2020   
814795              09/06/2020   
814796              09/06/2020   

                                            Tweet Content Tweet Location  
0       Why .Indians to prayer Iran a polluted http://...          Bihar  
1       Even death is âMade in Chinaâ now #CoronaV...            Goa  
2       One of the most widely read  published stories...            Goa  
3       Hahahh  RAW mission in Iran is failed due to C...         Punjab  
4       #BreakingNews: in show @sawaalwithamber on @SA...         Punjab  
...                                                   ...            ...  
814792  How to use contact lense

In [6]:
stop = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
              "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
              "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these",
              "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do",
              "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while",
              "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before",
              "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
              "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
              "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",
              "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [7]:
import string
#lowerCase
fr['Tweet Content'] = fr['Tweet Content'].map(lambda x:str(x).lower())

#user mention removes
fr['Tweet Content'] = fr['Tweet Content'].str.replace(r'@\S+', '')

#removes reTweet Content & cc
fr['Tweet Content'] = fr['Tweet Content'].str.replace(r'rt|cc', '')

#remove special characters except [a-zA-Z]
fr['Tweet Content'] = fr['Tweet Content'].map(lambda x:re.sub('[^a-zA-Z]',' ',str(x)))

#remove link starts with https
fr['Tweet Content'] = fr['Tweet Content'].map(lambda x:re.sub('http.*','',str(x)))

fr['Tweet Content'] = fr['Tweet Content'].str.replace(r'(https|http)?:\/(\w|\.|\/|\?|\=|\&|\%)*\b','')

#punctuation
fr['Tweet Content'] = fr['Tweet Content'].str.replace('[{}]'.format(string.punctuation), '')


fr


,Tweet Posted Time (UTC),Tweet Content,Tweet Location
0,01/03/2020,why indians to prayer iran a polluted,Bihar
1,01/03/2020,even death is made in china now coronav...,Goa
2,01/03/2020,one of the most widely read published stories...,Goa
3,01/03/2020,hahahh raw mission in iran is failed due to c...,Punjab
4,01/03/2020,breakingnews in show on special assistant ...,Punjab
...,...,...,...
814792,09/06/2020,how to use contact lenses safely during these ...,Dehradun
814793,09/06/2020,digital justice system in india overview re...,Dehradun
814794,09/06/2020,indiafightscorona stigma around covid can ...,Dehradun
814795,09/06/2020,these steps will help revive indianeconomy ...,Dehradun


In [8]:
state_list  = fr['Tweet Location'].unique()
state_set  = set(state_list)
state_set

{'Ahmedabad',
 'Andaman and Nicobar Islands',
 'Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bangalore',
 'Bhubaneswar',
 'Bihar',
 'Chandigarh',
 'Chennai',
 'Chhattisgarh',
 'Dadra and Nagar Haveli',
 'Daman and Diu',
 'Dehradun',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Hyderabad',
 'Indore',
 'Jaipur',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Kochi',
 'Kolkata',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Mumbai',
 'Nagaland',
 'Orissa',
 'Pondicherry',
 'Pune',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'Tirupati',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'Visakhapatnam',
 'West Bengal'}

In [9]:
fr['Tweet Posted Time (UTC)'].unique()

array(['01/03/2020', '02/03/2020', '03/03/2020', '04/03/2020',
       '05/03/2020', '06/03/2020', '07/03/2020', '08/03/2020',
       '09/03/2020', '10/03/2020', '11/03/2020', '12/03/2020',
       '13/03/2020', '14/03/2020', '15/03/2020', '16/03/2020',
       '17/03/2020', '18/03/2020', '19/03/2020', '20/03/2020',
       '21/03/2020', '22/03/2020', '23/03/2020', '24/03/2020',
       '25/03/2020', '26/03/2020', '27/03/2020', '28/03/2020',
       '29/03/2020', '30/03/2020', '31/03/2020', '01/04/2020',
       '02/04/2020', '03/04/2020', '04/04/2020', '05/04/2020',
       '06/04/2020', '07/04/2020', '08/04/2020', '09/04/2020',
       '10/04/2020', '11/04/2020', '12/04/2020', '13/04/2020',
       '14/04/2020', '15/04/2020', '16/04/2020', '17/04/2020',
       '18/04/2020', '19/04/2020', '20/04/2020', '21/04/2020',
       '22/04/2020', '23/04/2020', '24/04/2020', '25/04/2020',
       '26/04/2020', '27/04/2020', '28/04/2020', '29/04/2020',
       '30/04/2020', '01/05/2020', '02/05/2020', '03/05

In [10]:
for st in state_set:
    location = fr.loc[fr['Tweet Location']== st ]
    location.to_csv(str(st) + ".csv" , index = True)

In [11]:
df = pd.read_csv("assam.csv")

In [12]:
print(df)

     Unnamed: 0 Tweet Posted Time (UTC)  \
0        216956              23/04/2020   
1        216957              23/04/2020   
2        216958              23/04/2020   
3        216959              23/04/2020   
4        216960              23/04/2020   
..          ...                     ...   
927      809501              09/06/2020   
928      809502              09/06/2020   
929      809503              09/06/2020   
930      809504              09/06/2020   
931      809505              09/06/2020   

                                         Tweet Content Tweet Location  
0    lots of love and greetings for this holy and f...          Assam  
1    wrong analogy  harsh  road aidents doesn t mul...          Assam  
2    rather to restrict inter district transpo  stu...          Assam  
3    i created a live  covid   stats tracking websi...          Assam  
4    une bonne question   covid   pandemic in cambo...          Assam  
..                                                 

In [13]:
df["Tweet Posted Time (UTC)"]= pd.to_datetime(df["Tweet Posted Time (UTC)"], format = "%d/%m/%Y")
df

,Unnamed: 0,Tweet Posted Time (UTC),Tweet Content,Tweet Location
0,216956,2020-04-23,lots of love and greetings for this holy and f...,Assam
1,216957,2020-04-23,wrong analogy harsh road aidents doesn t mul...,Assam
2,216958,2020-04-23,rather to restrict inter district transpo stu...,Assam
3,216959,2020-04-23,i created a live covid stats tracking websi...,Assam
4,216960,2020-04-23,une bonne question covid pandemic in cambo...,Assam
...,...,...,...,...
927,809501,2020-06-09,i congratulate newzeeland govt for their suess...,Assam
928,809502,2020-06-09,icmr approves diphu medical college as covid ...,Assam
929,809503,2020-06-09,unlocking has been staed but keep in mind that...,Assam
930,809504,2020-06-09,covid free new zealand lifts curbs at midni...,Assam


In [14]:
y1 = "25-03-2020"
y2 = "14-04-2020"
y3 = "15-04-2020"
y4 = "03-05-2020"
y5 = "4-05-2020"
y6 = "17-05-2020"
y7 = "18-05-2020"
y8 = "31-05-2020"
pd.to_datetime(y4, format = "%d-%m-%Y")

Timestamp('2020-05-03 00:00:00')

In [15]:
lock1 = df[ ( df['Tweet Posted Time (UTC)'] > pd.to_datetime(y1, format = "%d-%m-%Y")) & (df['Tweet Posted Time (UTC)'] < pd.to_datetime(y2, format = "%d-%m-%Y")) ]
lock2 = df[ ( df['Tweet Posted Time (UTC)'] > pd.to_datetime(y3, format = "%d-%m-%Y")) & (df['Tweet Posted Time (UTC)'] < pd.to_datetime(y4, format = "%d-%m-%Y")) ]
lock3 = df[ ( df['Tweet Posted Time (UTC)'] > pd.to_datetime(y5, format = "%d-%m-%Y")) & (df['Tweet Posted Time (UTC)'] < pd.to_datetime(y6, format = "%d-%m-%Y")) ]
lock4 = df[ ( df['Tweet Posted Time (UTC)'] > pd.to_datetime(y7, format = "%d-%m-%Y")) & (df['Tweet Posted Time (UTC)'] < pd.to_datetime(y8, format = "%d-%m-%Y")) ]
lock2.to_csv("lockdown1.csv" , index = True)

In [16]:
def lockdown(state):
    df = pd.read_csv(state + ".csv")
    df["Tweet Posted Time (UTC)"]= pd.to_datetime(df["Tweet Posted Time (UTC)"], format = "%d/%m/%Y")
    lock1 = df[ ( df['Tweet Posted Time (UTC)'] > pd.to_datetime(y1, format = "%d-%m-%Y")) & (df['Tweet Posted Time (UTC)'] < pd.to_datetime(y2, format = "%d-%m-%Y")) ]
    lock2 = df[ ( df['Tweet Posted Time (UTC)'] > pd.to_datetime(y3, format = "%d-%m-%Y")) & (df['Tweet Posted Time (UTC)'] < pd.to_datetime(y4, format = "%d-%m-%Y")) ]
    lock3 = df[ ( df['Tweet Posted Time (UTC)'] > pd.to_datetime(y5, format = "%d-%m-%Y")) & (df['Tweet Posted Time (UTC)'] < pd.to_datetime(y6, format = "%d-%m-%Y")) ]
    lock4 = df[ ( df['Tweet Posted Time (UTC)'] > pd.to_datetime(y7, format = "%d-%m-%Y")) & (df['Tweet Posted Time (UTC)'] < pd.to_datetime(y8, format = "%d-%m-%Y")) ]
    lock1.to_csv("lockdown" + state + "1.csv" , index = True)
    lock2.to_csv("lockdown" + state + "2.csv" , index = True)
    lock3.to_csv("lockdown" + state + "3.csv" , index = True)
    lock4.to_csv("lockdown" + state + "4.csv" , index = True)

In [17]:
for st in state_set:
    lockdown(str(st))